In [2]:
# DSPy demo goes here
# see https://github.com/ericmelz/dspy/blob/main/intro.ipynb

In [9]:
import dspy
import openai

In [10]:

f = open('/Users/ericmelz/Documents/code/openai-quickstart-python-emelz/.env', 'r')

KEY='OPENAI_API_KEY'
l=f.readline()
api_key = 'undefined!'
while l:
    if l.startswith(KEY):
        print('FOUND')
        api_key = l[len(KEY)+1:-1]
        break
    l = f.readline()

f.close()

openai.api_key=api_key

FOUND


In [4]:
turbo = dspy.OpenAI(model='gpt-3.5-turbo')
colbertv2_wiki17_abstracts = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')

dspy.settings.configure(lm=turbo, rm=colbertv2_wiki17_abstracts)

In [5]:
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

In [6]:
# Define the predictor.
generate_answer = dspy.Predict(BasicQA)

In [13]:
# Call the predictor on a particular input.
pred = generate_answer(question='What is the capital of France?')

In [14]:
pred

Prediction(
    answer='Paris'
)

## Some math using BasicQA and turbo

In [38]:
def expect(gold, pred):
    if str(gold) == str(pred.answer):
        print('correct')
    else:
        print(F'INCORRECT.  Expected {gold}, got {pred.answer}')

In [27]:
pred = generate_answer(question='2+2')
expect(4, pred)

correct


In [28]:
pred = generate_answer(question='7+9')
expect(16, pred)

correct


In [31]:
pred = generate_answer(question='27+59')
expect(86, pred)

correct


In [33]:
pred = generate_answer(question='327+959')
expect(1286, pred)

correct


In [34]:
pred = generate_answer(question='4327+8959')
expect(13286, pred)

correct


In [35]:
pred = generate_answer(question='4329+8959')
expect(13288, pred)

correct


In [36]:
pred = generate_answer(question='44329+28959')
expect(73288, pred)

correct


In [39]:
pred = generate_answer(question='944329+928959')
expect(1873288, pred)

INCORRECT.  Expected 1873288, got 1876288


## CommonsenseQA
See https://github.com/ezelikman/STaR/blob/main/commonsenseqa/prompts.txt

In [44]:
q = """
Q: What do people use to absorb extra ink from a fountain pen?
Answer Choices:
(a) shirt pocket
(b) calligrapher's hand
(c) inkwell
(d) desk drawer
(e) blotter
"""

pred = generate_answer(question=q)
expect('(e) blotter', pred)

correct


In [46]:
q = """
Q: What home entertainment equipment requires cable?
Answer Choices:
(a) radio shack
(b) substation
(c) television
(d) cabinet
(e) desk
"""
a = "(c) television"

expect(a, generate_answer(question=q))

INCORRECT.  Expected (c) television, got television


In [53]:
preamble = """
Answer the following multiple choice question.

Present the answer in the format
(${letter}) ${answer}

----
Example:

Q: What do people use to absorb extra ink from a fountain pen?
Answer Choices:
(a) shirt pocket
(b) calligrapher's hand
(c) inkwell
(d) desk drawer
(e) blotter

A: (e) blotter
----
"""

In [54]:
q = f"""
{preamble}
Q: What home entertainment equipment requires cable?
Answer Choices:
(a) radio shack
(b) substation
(c) television
(d) cabinet
(e) desk

A:
"""
a = "(c) television"

expect(a, generate_answer(question=q))

correct


In [55]:
q = f"""
{preamble}
Q: The fox walked from the city into the forest, what was it looking for?
Answer Choices:
(a) pretty flowers
(b) hen house
(c) natural habitat
(d) storybook
(e) dense forest

A:
"""
a = "(c) natural habitat"

expect(a, generate_answer(question=q))

correct


In [56]:
q = f"""
{preamble}
Q: Sammy wanted to go to where the people were. Where might he go?
Answer Choices:
(a) populated areas
(b) race track
(c) desert
(d) apartment
(e) roadblock

A:
"""
a = "(a) populated areas"

expect(a, generate_answer(question=q))

correct


In [57]:
q = f"""
{preamble}
Q: Where do you put your grapes just before checking out?
Answer Choices:
(a) mouth
(b) grocery cart
(c) super market
(d) fruit basket
(e) fruit market

A:
"""
a = "(b) grocery cart"

expect(a, generate_answer(question=q))

correct


In [59]:
q = f"""
{preamble}
Q: Google Maps and other highway and street GPS services have replaced what?
Answer Choices:
(a) united states
(b) mexico
(c) countryside
(d) atlas
(e) oceans

A:
"""
a = "(d) atlas"

expect(a, generate_answer(question=q))

correct


In [60]:
q = f"""
{preamble}
Q: Before getting a divorce, what did the wife feel who was doing all the work?
Answer Choices:
(a) harder
(b) anguish
(c) bitterness
(d) tears
(e) sadness

A:
"""
a = "(c) bitterness"

expect(a, generate_answer(question=q))

INCORRECT.  Expected (c) bitterness, got (a) harder


## Grade School Math
See https://github.com/ezelikman/STaR/blob/main/gsm/dev_rand_split.jsonl

In [62]:
qs = [
    {"question": "Janet\u2019s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?", "answer": "Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer\u2019s market.\n#### 18"},
    {"question": "A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?", "answer": "It takes 2/2=<<2/2=1>>1 bolt of white fiber\nSo the total amount of fabric is 2+1=<<2+1=3>>3 bolts of fabric\n#### 3"}
]

In [63]:
qs[0]['question']

"Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"

In [66]:
expect("18", generate_answer(question=qs[0]['question']))

INCORRECT.  Expected 18, got $24


In [67]:
qs[0]['answer']

'Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\n#### 18'

In [73]:
preamble = """
Answer the following question.  Use the format:

Question:
${question}

Rationale:
${rationale}

Answer:
${answer}
---
Question:
Janet’s ducks lay 18 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?

Rationale:
Janet sells 18 - 3 - 4 = <<18-3-4=11>>11 duck eggs a day.\nShe makes 11 * 2 = $<<11*2=22>>22 every day at the farmer’s market.\n#### 22

Answer:
22
---
Question:
Janet’s ducks lay 10 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $3 per fresh duck egg. How much in dollars does she make every day at the farmers' market?

Rationale:
Janet sells 10 - 3 - 4 = <<10-3-4=3>>3 duck eggs a day.\nShe makes 3 * 3 = $<<3*3=9>>9 every day at the farmer’s market.\n#### 9

Answer:
9
---
Question:
"""

In [74]:
q = f"""
{preamble}
{qs[0]['question']}
"""

In [75]:
print(q)



Answer the following question.  Use the format:

Question:
${question}

Rationale:
${rationale}

Answer:
${answer}
---
Question:
Janet’s ducks lay 18 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?

Rationale:
Janet sells 18 - 3 - 4 = <<18-3-4=11>>11 duck eggs a day.
She makes 11 * 2 = $<<11*2=22>>22 every day at the farmer’s market.
#### 22

Answer:
22
---
Question:
Janet’s ducks lay 10 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $3 per fresh duck egg. How much in dollars does she make every day at the farmers' market?

Rationale:
Janet sells 10 - 3 - 4 = <<10-3-4=3>>3 duck eggs a day.
She makes 3 * 3 = $<<3*3=9>>9 every day at the farmer’s market.
###

In [76]:
expect("18", generate_answer(question=q))

INCORRECT.  Expected 18, got 22


In [77]:
generate_answer(question=q)

Prediction(
    answer='22'
)

In [78]:
s = """
a
b
"""

In [79]:
s

'\na\nb\n'

In [81]:
qs[1]

{'question': 'A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?',
 'answer': 'It takes 2/2=<<2/2=1>>1 bolt of white fiber\nSo the total amount of fabric is 2+1=<<2+1=3>>3 bolts of fabric\n#### 3'}

In [86]:
q=qs[1]['question']

In [87]:
generate_answer(question=q)

Prediction(
    answer='3 bolts'
)

In [88]:
turbo.inspect_history(n=1)





Answer questions with short factoid answers.

---

Follow the following format.

Question: ${question}
Answer: often between 1 and 5 words

---

Question: A robe takes 2 bolts of blue fiber and half that much white fiber. How many bolts in total does it take?
Answer: 3 bolts





In [89]:
turbo.model

AttributeError: 'GPT3' object has no attribute 'model'

In [90]:
turbo.__dict__

{'kwargs': {'temperature': 0.0,
  'max_tokens': 150,
  'top_p': 1,
  'frequency_penalty': 0,
  'presence_penalty': 0,
  'n': 1,
  'model': 'gpt-3.5-turbo'},
 'provider': 'openai',
 'history': [{'prompt': 'Answer questions with short factoid answers.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nAnswer: often between 1 and 5 words\n\n---\n\nQuestion: What is the capital of France?\nAnswer:',
   'response': <OpenAIObject chat.completion id=chatcmpl-8DLAz5bCyE2htFoJQSWsg0Kpx5JD8 at 0x29a34f400> JSON: {
     "id": "chatcmpl-8DLAz5bCyE2htFoJQSWsg0Kpx5JD8",
     "object": "chat.completion",
     "created": 1698190357,
     "model": "gpt-3.5-turbo-0613",
     "choices": [
       {
         "index": 0,
         "message": {
           "role": "assistant",
           "content": "Paris"
         },
         "finish_reason": "stop"
       }
     ],
     "usage": {
       "prompt_tokens": 49,
       "completion_tokens": 1,
       "total_tokens": 50
     }
   },
   'kwargs': {'st

In [93]:
turbo.kwargs['model']

'gpt-3.5-turbo'

In [94]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ]
)

In [95]:
response

<OpenAIObject chat.completion id=chatcmpl-8DMoZpC4spzm5cpC66yUikQEX60ls at 0x29a43d6d0> JSON: {
  "id": "chatcmpl-8DMoZpC4spzm5cpC66yUikQEX60ls",
  "object": "chat.completion",
  "created": 1698196655,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The World Series in 2020 was played at Globe Life Field in Arlington, Texas."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 53,
    "completion_tokens": 18,
    "total_tokens": 71
  }
}

In [96]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Josh decides to try flipping a house.  He buys a house for $80,000 and then puts in $50,000 in repairs.  This increased the value of the house by 150%.  How much profit did he make?"},
    ]
)

In [97]:
response

<OpenAIObject chat.completion id=chatcmpl-8DMr3CslpMG8Dc01eW5S3nD4SRxAh at 0x29a592180> JSON: {
  "id": "chatcmpl-8DMr3CslpMG8Dc01eW5S3nD4SRxAh",
  "object": "chat.completion",
  "created": 1698196809,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The repair increased the value of the house by 80,000*.15=$<<80000*.15=12000>>12,000\nSo the total value of the house is now 80,000+12,000=$<<80000+12000=92000>>92,000\nThat means he made a profit of 92,000-80,000=$<<92000-80000=12000>>12,000. Answer: \\boxed{12000}."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 56,
    "completion_tokens": 99,
    "total_tokens": 155
  }
}